Trabajo con Datos con el Core: Actualización y Borrado de Filas
=====

* Última actualización: 12 de septiembre de 2022.

Preparación
---

In [1]:
from sqlalchemy import (
    Column,
    ForeignKey,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    select,
)

metadata_obj = MetaData()

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)


address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),  # Clave foranea
    Column("email_address", String, nullable=False),
)

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=True,
    future=True,
)

metadata_obj.create_all(engine)

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "spongebob", "fullname": "Spongebob Squarepants"},
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()


select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)

with engine.connect() as conn:
    result = conn.execute(insert_stmt)
    conn.commit()

2022-09-13 22:01:33,693 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:01:33,693 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-13 22:01:33,694 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:01:33,694 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-13 22:01:33,695 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:01:33,696 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-13 22:01:33,696 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:01:33,697 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-13 22:01:33,698 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:01:33,699 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-13 22:01:33,699 INFO sqlalchemy.engine.Engine [no key 0.00048s] ()
2022-09-13 22:01:33,700 INFO sqlalchemy.engine.Engine 
C

UPDATE
---

In [2]:
#
# Uso básico
#
from sqlalchemy import update

stmt = (
    update(user_table)
    .where(user_table.c.name == "patrick")
    .values(fullname="Patrick the Star")
)
print(stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1


In [3]:
#
# Soporta la mayor parte de formas de UPDATE en SQL
#
stmt = update(user_table).values(fullname="Username: " + user_table.c.name)
print(stmt)

UPDATE user_account SET fullname=(:name_1 || user_account.name)


In [4]:
#
# Update con executemany()
#
from sqlalchemy import bindparam

stmt = (
    update(user_table)
    .where(user_table.c.name == bindparam("oldname"))
    .values(name=bindparam("newname"))
)
with engine.begin() as conn:
    conn.execute(
        stmt,
        [
            {"oldname": "jack", "newname": "ed"},
            {"oldname": "wendy", "newname": "mary"},
            {"oldname": "jim", "newname": "jake"},
        ],
    )

2022-09-13 22:01:33,958 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:01:33,960 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.name = ?
2022-09-13 22:01:33,961 INFO sqlalchemy.engine.Engine [generated in 0.00094s] (('ed', 'jack'), ('mary', 'wendy'), ('jake', 'jim'))
2022-09-13 22:01:33,962 INFO sqlalchemy.engine.Engine COMMIT


Updates correlacionados
---

In [5]:
scalar_subq = (
    select(address_table.c.email_address)
    .where(address_table.c.user_id == user_table.c.id)
    .order_by(address_table.c.id)
    .limit(1)
    .scalar_subquery()
)
update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)

UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


UPDATE ... FROM
---

In [6]:
update_stmt = (
    update(user_table)
    .where(user_table.c.id == address_table.c.user_id)
    .where(address_table.c.email_address == "patrick@aol.com")
    .values(fullname="Pat")
)

print(update_stmt)

UPDATE user_account SET fullname=:fullname FROM address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


In [7]:
#
# Actualización de varias tablas
#
update_stmt = (
    update(user_table)
    .where(user_table.c.id == address_table.c.user_id)
    .where(address_table.c.email_address == "patrick@aol.com")
    .values(
        {user_table.c.fullname: "Pat", address_table.c.email_address: "pat@aol.com"}
    )
)
from sqlalchemy.dialects import mysql

print(update_stmt.compile(dialect=mysql.dialect()))

UPDATE user_account, address SET address.email_address=%s, user_account.fullname=%s WHERE user_account.id = address.user_id AND address.email_address = %s


Update con parámetros
----

In [8]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [
            {"x": 1, "y": 1},
            {"x": 2, "y": 4},
        ],
    )
    conn.commit()

some_table = Table(
    "some_table",
    metadata_obj,
    autoload_with=engine,
)

2022-09-13 22:01:34,421 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:01:34,422 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-09-13 22:01:34,423 INFO sqlalchemy.engine.Engine [generated in 0.00192s] ()
2022-09-13 22:01:34,424 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-09-13 22:01:34,425 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ((1, 1), (2, 4))
2022-09-13 22:01:34,426 INFO sqlalchemy.engine.Engine COMMIT
2022-09-13 22:01:34,427 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:01:34,427 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-09-13 22:01:34,428 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:01:34,429 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-09-13 22:01:34,430 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-09-13 

In [9]:
update_stmt = update(some_table).ordered_values(
    (some_table.c.y, 20), (some_table.c.x, some_table.c.y + 10)
)
print(update_stmt)

UPDATE some_table SET y=:y, x=(some_table.y + :y_1)


Expresión DELETE
---

In [10]:
#
# Borrado de una tabla
#
from sqlalchemy import delete

stmt = delete(user_table).where(user_table.c.name == "patrick")
print(stmt)

DELETE FROM user_account WHERE user_account.name = :name_1


In [11]:
#
# Borrado de varias tablas
#
delete_stmt = (
    delete(user_table)
    .where(user_table.c.id == address_table.c.user_id)
    .where(address_table.c.email_address == "patrick@aol.com")
)

from sqlalchemy.dialects import mysql

print(delete_stmt.compile(dialect=mysql.dialect()))

DELETE FROM user_account USING user_account, address WHERE user_account.id = address.user_id AND address.email_address = %s


Conteo de tablas afectadas con UPDATE y DELETE
----

In [12]:
with engine.begin() as conn:
    result = conn.execute(
        update(user_table)
        .values(fullname="Patrick McStar")
        .where(user_table.c.name == "patrick")
    )
    print(result.rowcount)

2022-09-13 22:01:34,735 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:01:34,737 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2022-09-13 22:01:34,738 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ('Patrick McStar', 'patrick')
1
2022-09-13 22:01:34,739 INFO sqlalchemy.engine.Engine COMMIT


Uso de RETURNING con UPDATE y DELETE
---

In [13]:
update_stmt = (
    update(user_table)
    .where(user_table.c.name == "patrick")
    .values(fullname="Patrick the Star")
    .returning(user_table.c.id, user_table.c.name)
)
print(update_stmt)
delete_stmt = (
    delete(user_table)
    .where(user_table.c.name == "patrick")
    .returning(user_table.c.id, user_table.c.name)
)
print(delete_stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name
DELETE FROM user_account WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name
